In [1]:
import os
import sys
import argparse
import time

import torch.utils.model_zoo as model_zoo
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
from torch.utils.tensorboard import SummaryWriter

import datasets
from model import L2CS
from utils import select_device

In [2]:
!nvidia-smi

Tue Jun  7 03:40:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:10:1C.0 Off |                    0 |
| N/A   34C    P0    52W / 400W |      3MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:10:1D.0 Off |                    0 |
| N/A   

In [3]:
!ls /project/data/Label

p00.label  p03.label  p06.label  p09.label  p12.label
p01.label  p04.label  p07.label  p10.label  p13.label
p02.label  p05.label  p08.label  p11.label  p14.label


In [4]:
!ls /project/data/Image

p00  p01  p02  p03  p04  p05  p06  p07	p08  p09  p10  p11  p12  p13  p14


In [5]:
def parse_args():
    """Parse input arguments."""
    parser = argparse.ArgumentParser(description='Gaze estimation using L2CSNet.')
#     # Gaze360
#     parser.add_argument(
#         '--gaze360image_dir', dest='gaze360image_dir', help='Directory path for gaze images.',
#         default='datasets/Gaze360/Image', type=str)
#     parser.add_argument(
#         '--gaze360label_dir', dest='gaze360label_dir', help='Directory path for gaze labels.',
#         default='datasets/Gaze360/Label/train.label', type=str)
#     # mpiigaze
    parser.add_argument(
        '--gazeMpiimage_dir', dest='gazeMpiimage_dir', help='Directory path for gaze images.',
        default='/project/data/Image', type=str)
    parser.add_argument(
        '--gazeMpiilabel_dir', dest='gazeMpiilabel_dir', help='Directory path for gaze labels.',
        default='/project/data/Label', type=str)

    # Important args -------------------------------------------------------------------------------------------------------
    # ----------------------------------------------------------------------------------------------------------------------
    parser.add_argument(
        '--dataset', dest='dataset', help='mpiigaze, rtgene, gaze360, ethgaze',
        default= "gaze360", type=str)
    parser.add_argument(
        '--output', dest='output', help='Path of output models.',
        default='output/snapshots/', type=str)
    parser.add_argument(
        '--snapshot', dest='snapshot', help='Path of model snapshot.',
        default='', type=str)
    parser.add_argument(
        '--gpu', dest='gpu_id', help='GPU device id to use [0] or multiple 0,1,2,3',
        default='0', type=str)
    parser.add_argument(
        '--num_epochs', dest='num_epochs', help='Maximum number of training epochs.',
        default=60, type=int)
    parser.add_argument(
        '--batch_size', dest='batch_size', help='Batch size.',
        default=1, type=int)
    parser.add_argument(
        '--arch', dest='arch', help='Network architecture, can be: ResNet18, ResNet34, [ResNet50], ''ResNet101, ResNet152, Squeezenet_1_0, Squeezenet_1_1, MobileNetV2',
        default='ResNet50', type=str)
    parser.add_argument(
        '--alpha', dest='alpha', help='Regression loss coefficient.',
        default=1, type=float)
    parser.add_argument(
        '--lr', dest='lr', help='Base learning rate.',
        default=0.00001, type=float)
    # ---------------------------------------------------------------------------------------------------------------------
    # Important args ------------------------------------------------------------------------------------------------------
    args = parser.parse_args()
    return args

In [6]:
class Nothing:
    pass
args = Nothing()
args.gazeMpiimage_dir = '/project/data/Image'
args.gazeMpiilabel_dir = '/project/data/Label'
args.output = '/project/results/output/snapshots/'
args.dataset = 'mpiigaze'
args.snapshot=''
args.gpu_id = '0,1,2,3'
args.num_epochs = 60
args.batch_size = 100
args.arch = 'ResNet50'
args.alpha = 1.0
args.lr = 0.00001



In [7]:
def get_ignored_params(model):
    # Generator function that yields ignored params.
    b = [model.conv1, model.bn1, model.fc_finetune]
    for i in range(len(b)):
        for module_name, module in b[i].named_modules():
            if 'bn' in module_name:
                module.eval()
            for name, param in module.named_parameters():
                yield param

def get_non_ignored_params(model):
    # Generator function that yields params that will be optimized.
    b = [model.layer1, model.layer2, model.layer3, model.layer4]
    for i in range(len(b)):
        for module_name, module in b[i].named_modules():
            if 'bn' in module_name:
                module.eval()
            for name, param in module.named_parameters():
                yield param

def get_fc_params(model):
    # Generator function that yields fc layer params.
    b = [model.fc_yaw_gaze, model.fc_pitch_gaze]
    for i in range(len(b)):
        for module_name, module in b[i].named_modules():
            for name, param in module.named_parameters():
                yield param
                
def load_filtered_state_dict(model, snapshot):
    # By user apaszke from discuss.pytorch.org
    model_dict = model.state_dict()
    snapshot = {k: v for k, v in snapshot.items() if k in model_dict}
    model_dict.update(snapshot)
    model.load_state_dict(model_dict)


def getArch_weights(arch, bins):
    if arch == 'ResNet18':
        model = L2CS(torchvision.models.resnet.BasicBlock, [2, 2, 2, 2], bins)
        pre_url = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'
    elif arch == 'ResNet34':
        model = L2CS(torchvision.models.resnet.BasicBlock, [3, 4, 6, 3], bins)
        pre_url = 'https://download.pytorch.org/models/resnet34-333f7ec4.pth'
    elif arch == 'ResNet101':
        model = L2CS(torchvision.models.resnet.Bottleneck, [3, 4, 23, 3], bins)
        pre_url = 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth'
    elif arch == 'ResNet152':
        model = L2CS(torchvision.models.resnet.Bottleneck,[3, 8, 36, 3], bins)
        pre_url = 'https://download.pytorch.org/models/resnet152-b121ed2d.pth'
    else:
        if arch != 'ResNet50':
            print('Invalid value for architecture is passed! '
                  'The default value of ResNet50 will be used instead!')
        model = L2CS(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], bins)
        pre_url = 'https://download.pytorch.org/models/resnet50-19c8e357.pth'

    return model, pre_url

In [8]:
# args = parse_args()
cudnn.enabled = True
num_epochs = args.num_epochs
batch_size = args.batch_size

In [9]:
gpu = select_device(args.gpu_id, batch_size=args.batch_size)
print(gpu)

cuda:0


In [10]:
data_set=args.dataset
alpha = args.alpha
output=args.output

transformations = transforms.Compose([
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

['p00.label', 'p01.label', 'p02.label', 'p03.label', 'p04.label', 'p05.label', 'p06.label', 'p07.label', 'p08.label', 'p09.label', 'p10.label', 'p11.label', 'p12.label', 'p13.label', 'p14.label']


In [11]:
#for tensorboard graphs
folder = os.listdir(args.gazeMpiilabel_dir)
folder.sort()
testlabelpathcombined = [os.path.join(args.gazeMpiilabel_dir, j) for j in folder]
fold = 0

dataset=datasets.Mpiigaze(testlabelpathcombined,args.gazeMpiimage_dir, transformations, True, 180, fold)
train_loader_gaze = DataLoader(
    dataset=dataset,
    batch_size=int(batch_size),
    shuffle=True,
    num_workers=4,
    pin_memory=True)
examples = iter(train_loader_gaze)
example_images,labels_gaze, cont_labels_gaze, name = examples.next()

0 items removed from dataset that have an angle > 180


In [12]:
%%time
start = time.time()

folder = os.listdir(args.gazeMpiilabel_dir)
folder.sort()
testlabelpathcombined = [os.path.join(args.gazeMpiilabel_dir, j) for j in folder]
for fold in range(15):
    model, pre_url = getArch_weights(args.arch, 28)
    print(fold, model.conv1)
    load_filtered_state_dict(model, model_zoo.load_url(pre_url))
#     model = nn.DataParallel(model)
#     model.to(gpu)
#     print('Again, model.conv1', model.conv1)
    print('Loading data.')
    dataset=datasets.Mpiigaze(testlabelpathcombined,args.gazeMpiimage_dir, transformations, True, 180, fold)
    
    train_loader_gaze = DataLoader(
        dataset=dataset,
        batch_size=int(batch_size),
        shuffle=True,
        num_workers=4,
        pin_memory=True)
    
    torch.backends.cudnn.benchmark = True
    
    fold_path = os.path.join(output, 'fold' + f'{fold:0>2}'+'/')
    print(f"fold_path is {fold_path}")
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)
        
    writer =SummaryWriter(fold_path)
    example_images = Variable(example_images)
    writer.add_graph(model, example_images)
        
#     summary_name = '{}_{}'.format('L2CS-mpiigaze', int(time.time()))
# #     print('summary_name=', summary_name)

#     if not os.path.exists(os.path.join(output+'/{}'.format(summary_name),'fold' + str(fold))):
#         os.makedirs(os.path.join(output+'/{}'.format(summary_name),'fold' + str(fold)))

    
    criterion = nn.CrossEntropyLoss().cuda(gpu)
    reg_criterion = nn.MSELoss().cuda(gpu)
    softmax = nn.Softmax(dim=1).cuda(gpu)
    idx_tensor = [idx for idx in range(28)]
    idx_tensor = Variable(torch.FloatTensor(idx_tensor)).cuda(gpu)

#     print('Again, model.conv1', model.conv1)
    # Optimizer gaze
    #### origianally wrong number of arguments
    optimizer_gaze = torch.optim.Adam([
        {'params': get_ignored_params(model), 'lr': 0}, 
        {'params': get_non_ignored_params(model), 'lr': args.lr},
        {'params': get_fc_params(model), 'lr': args.lr}
    ], args.lr)

    
    
    configuration = f"\ntrain configuration, gpu_id={args.gpu_id}, batch_size={batch_size}, model_arch={args.arch}\n Start training dataset={data_set}, loader={len(train_loader_gaze)}, fold={fold}--------------\n"
#     print(configuration)
    model.to(gpu)
    model = nn.DataParallel(model, device_ids=[0,1,2,3])
    
    for epoch in range(num_epochs):
        sum_loss_pitch_gaze = sum_loss_yaw_gaze = iter_gaze = 0

        for i, (images_gaze, labels_gaze, cont_labels_gaze,name) in enumerate(train_loader_gaze):
            images_gaze = Variable(images_gaze).cuda(gpu)

            # Binned labels
            label_pitch_gaze = Variable(labels_gaze[:, 0]).cuda(gpu)
            label_yaw_gaze = Variable(labels_gaze[:, 1]).cuda(gpu)

            # Continuous labels
            label_pitch_cont_gaze = Variable(cont_labels_gaze[:, 0]).cuda(gpu)
            label_yaw_cont_gaze = Variable(cont_labels_gaze[:, 1]).cuda(gpu)

            pitch, yaw = model(images_gaze)

            # Cross entropy loss
            loss_pitch_gaze = criterion(pitch, label_pitch_gaze)
            loss_yaw_gaze = criterion(yaw, label_yaw_gaze)

            # MSE loss
            pitch_predicted = softmax(pitch)
            yaw_predicted = softmax(yaw)

            # mapping from binned (0 to 28) to angels (-42 to 42) 
            pitch_predicted = \
                torch.sum(pitch_predicted * idx_tensor, 1) * 3 - 42
            yaw_predicted = \
                torch.sum(yaw_predicted * idx_tensor, 1) * 3 - 42

            loss_reg_pitch = reg_criterion(
                pitch_predicted, label_pitch_cont_gaze)
            loss_reg_yaw = reg_criterion(
                yaw_predicted, label_yaw_cont_gaze)

            
            # Total loss
            loss_pitch_gaze += alpha * loss_reg_pitch
            loss_yaw_gaze += alpha * loss_reg_yaw

            sum_loss_pitch_gaze += loss_pitch_gaze
            sum_loss_yaw_gaze += loss_yaw_gaze

            
            
            loss_seq = [loss_pitch_gaze, loss_yaw_gaze]
            grad_seq = \
                [torch.tensor(1.0).cuda(gpu) for _ in range(len(loss_seq))]

            optimizer_gaze.zero_grad(set_to_none=True)
            torch.autograd.backward(loss_seq, grad_seq)
            optimizer_gaze.step()

            iter_gaze += 1
            
            iterations = len(dataset)//batch_size
            div10 = iterations/10
            if (i+1) % div10 == 0:  #for every div10 batches
                now=time.time()
                elapsed = now-start
                
                yaw_loss = sum_loss_pitch_gaze/iter_gaze
                pitch_loss = sum_loss_yaw_gaze/iter_gaze
                print(f'Fold: {fold} Epoch [{epoch+1}/{num_epochs}], Iter [{i+1}/{len(dataset)//batch_size}] Losses: '
                        f'Gaze Yaw {yaw_loss:.4f},Gaze Pitch {pitch_loss:.3f}'
                         f' elapsed:{elapsed:.1f}s')
                


        # Save models at numbered epochs.
#         print("writing to tensor board...")
        writer.add_scalar('pitch_loss', pitch_loss, epoch)
        writer.add_scalar('yaw_loss', yaw_loss, epoch)
        
        if epoch % 1 == 0 and epoch < num_epochs:
            print(f"fold_path is still? {fold_path}")
            pathf = fold_path + 'epoch_' + str(epoch+1) + '.pkl'
            print(pathf)
            print('Taking snapshot...')
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer_gaze
                    .state_dict(),
                'pitch_loss': pitch_loss,
                'yaw_loss': yaw_loss
                }, pathf)
            
    writer.close()   #after each fold
            
#             torch.save(model.state_dict(), pathf) 
#                             output+'/fold' + str(fold) +'/'+
#                             'epoch_' + str(epoch+1) + '.pkl')
#                 )

      

0 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
Loading data.
0 items removed from dataset that have an angle > 180
fold_path is /project/results/output/snapshots/fold00/


/usr/local/lib/python3.8/dist-packages/torch/optim/adam.py:81: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(Adam, self).__init__(params, defaults)


Fold: 0 Epoch [1/60], Iter [42/420] Losses: Gaze Yaw 77.6128,Gaze Pitch 32.777 elapsed:79.4s
Fold: 0 Epoch [1/60], Iter [84/420] Losses: Gaze Yaw 69.3238,Gaze Pitch 29.254 elapsed:90.6s
Fold: 0 Epoch [1/60], Iter [126/420] Losses: Gaze Yaw 57.7450,Gaze Pitch 26.053 elapsed:101.7s
Fold: 0 Epoch [1/60], Iter [168/420] Losses: Gaze Yaw 47.9834,Gaze Pitch 23.930 elapsed:112.7s
Fold: 0 Epoch [1/60], Iter [210/420] Losses: Gaze Yaw 41.7535,Gaze Pitch 21.702 elapsed:123.8s
Fold: 0 Epoch [1/60], Iter [252/420] Losses: Gaze Yaw 36.8102,Gaze Pitch 20.133 elapsed:135.0s
Fold: 0 Epoch [1/60], Iter [294/420] Losses: Gaze Yaw 33.3299,Gaze Pitch 18.663 elapsed:145.7s
Fold: 0 Epoch [1/60], Iter [336/420] Losses: Gaze Yaw 30.9661,Gaze Pitch 17.621 elapsed:157.0s
Fold: 0 Epoch [1/60], Iter [378/420] Losses: Gaze Yaw 28.6843,Gaze Pitch 16.669 elapsed:168.1s
Fold: 0 Epoch [1/60], Iter [420/420] Losses: Gaze Yaw 26.8120,Gaze Pitch 15.902 elapsed:179.4s
fold_path is still? /project/results/output/snapshots/

Fold: 0 Epoch [8/60], Iter [378/420] Losses: Gaze Yaw 3.5783,Gaze Pitch 3.019 elapsed:841.3s
Fold: 0 Epoch [8/60], Iter [420/420] Losses: Gaze Yaw 3.6213,Gaze Pitch 3.018 elapsed:850.1s
fold_path is still? /project/results/output/snapshots/fold00/
/project/results/output/snapshots/fold00/epoch_8.pkl
Taking snapshot...
Fold: 0 Epoch [9/60], Iter [42/420] Losses: Gaze Yaw 3.2098,Gaze Pitch 3.039 elapsed:861.5s
Fold: 0 Epoch [9/60], Iter [84/420] Losses: Gaze Yaw 3.6077,Gaze Pitch 2.742 elapsed:870.6s
Fold: 0 Epoch [9/60], Iter [126/420] Losses: Gaze Yaw 3.5158,Gaze Pitch 2.938 elapsed:880.1s
Fold: 0 Epoch [9/60], Iter [168/420] Losses: Gaze Yaw 3.5109,Gaze Pitch 2.858 elapsed:889.2s
Fold: 0 Epoch [9/60], Iter [210/420] Losses: Gaze Yaw 3.3939,Gaze Pitch 2.779 elapsed:898.4s
Fold: 0 Epoch [9/60], Iter [252/420] Losses: Gaze Yaw 3.3398,Gaze Pitch 2.749 elapsed:907.5s
Fold: 0 Epoch [9/60], Iter [294/420] Losses: Gaze Yaw 3.2878,Gaze Pitch 2.822 elapsed:916.8s
Fold: 0 Epoch [9/60], Iter [336

Fold: 0 Epoch [16/60], Iter [210/420] Losses: Gaze Yaw 2.0735,Gaze Pitch 1.860 elapsed:1620.4s
Fold: 0 Epoch [16/60], Iter [252/420] Losses: Gaze Yaw 2.0583,Gaze Pitch 1.841 elapsed:1628.9s
Fold: 0 Epoch [16/60], Iter [294/420] Losses: Gaze Yaw 2.0515,Gaze Pitch 1.813 elapsed:1637.5s
Fold: 0 Epoch [16/60], Iter [336/420] Losses: Gaze Yaw 2.0680,Gaze Pitch 1.816 elapsed:1646.2s
Fold: 0 Epoch [16/60], Iter [378/420] Losses: Gaze Yaw 2.0711,Gaze Pitch 1.839 elapsed:1654.9s
Fold: 0 Epoch [16/60], Iter [420/420] Losses: Gaze Yaw 2.0910,Gaze Pitch 1.835 elapsed:1663.6s
fold_path is still? /project/results/output/snapshots/fold00/
/project/results/output/snapshots/fold00/epoch_16.pkl
Taking snapshot...
Fold: 0 Epoch [17/60], Iter [42/420] Losses: Gaze Yaw 2.0801,Gaze Pitch 1.370 elapsed:1674.8s
Fold: 0 Epoch [17/60], Iter [84/420] Losses: Gaze Yaw 1.8497,Gaze Pitch 1.386 elapsed:1683.2s
Fold: 0 Epoch [17/60], Iter [126/420] Losses: Gaze Yaw 1.7598,Gaze Pitch 1.436 elapsed:1691.5s
Fold: 0 Epoc

...

Fold: 14 Epoch [26/60], Iter [378/420] Losses: Gaze Yaw 1.2513,Gaze Pitch 1.119 elapsed:80090.5s
Fold: 14 Epoch [26/60], Iter [420/420] Losses: Gaze Yaw 1.2858,Gaze Pitch 1.146 elapsed:80098.4s
fold_path is still? /project/results/output/snapshots/fold14/
/project/results/output/snapshots/fold14/epoch_26.pkl
Taking snapshot...
Fold: 14 Epoch [27/60], Iter [42/420] Losses: Gaze Yaw 1.6142,Gaze Pitch 1.393 elapsed:80111.2s
Fold: 14 Epoch [27/60], Iter [84/420] Losses: Gaze Yaw 1.5417,Gaze Pitch 1.219 elapsed:80121.7s
Fold: 14 Epoch [27/60], Iter [126/420] Losses: Gaze Yaw 1.3749,Gaze Pitch 1.128 elapsed:80132.5s
Fold: 14 Epoch [27/60], Iter [168/420] Losses: Gaze Yaw 1.2834,Gaze Pitch 1.148 elapsed:80143.3s
Fold: 14 Epoch [27/60], Iter [210/420] Losses: Gaze Yaw 1.2284,Gaze Pitch 1.130 elapsed:80154.1s
Fold: 14 Epoch [27/60], Iter [252/420] Losses: Gaze Yaw 1.2225,Gaze Pitch 1.102 elapsed:80165.1s
Fold: 14 Epoch [27/60], Iter [294/420] Losses: Gaze Yaw 1.2607,Gaze Pitch 1.107 elapsed:801

Fold: 14 Epoch [34/60], Iter [126/420] Losses: Gaze Yaw 0.9918,Gaze Pitch 0.919 elapsed:80776.4s
Fold: 14 Epoch [34/60], Iter [168/420] Losses: Gaze Yaw 0.9814,Gaze Pitch 0.901 elapsed:80786.9s
Fold: 14 Epoch [34/60], Iter [210/420] Losses: Gaze Yaw 0.9707,Gaze Pitch 0.860 elapsed:80797.2s
Fold: 14 Epoch [34/60], Iter [252/420] Losses: Gaze Yaw 0.9635,Gaze Pitch 0.856 elapsed:80807.7s
Fold: 14 Epoch [34/60], Iter [294/420] Losses: Gaze Yaw 0.9765,Gaze Pitch 0.897 elapsed:80817.6s
Fold: 14 Epoch [34/60], Iter [336/420] Losses: Gaze Yaw 0.9890,Gaze Pitch 0.917 elapsed:80828.1s
Fold: 14 Epoch [34/60], Iter [378/420] Losses: Gaze Yaw 0.9833,Gaze Pitch 0.937 elapsed:80838.4s
Fold: 14 Epoch [34/60], Iter [420/420] Losses: Gaze Yaw 0.9939,Gaze Pitch 0.922 elapsed:80849.0s
fold_path is still? /project/results/output/snapshots/fold14/
/project/results/output/snapshots/fold14/epoch_34.pkl
Taking snapshot...
Fold: 14 Epoch [35/60], Iter [42/420] Losses: Gaze Yaw 1.3766,Gaze Pitch 0.880 elapsed:80

Fold: 14 Epoch [41/60], Iter [336/420] Losses: Gaze Yaw 0.9468,Gaze Pitch 0.744 elapsed:81454.0s
Fold: 14 Epoch [41/60], Iter [378/420] Losses: Gaze Yaw 0.9340,Gaze Pitch 0.746 elapsed:81462.3s
Fold: 14 Epoch [41/60], Iter [420/420] Losses: Gaze Yaw 0.9438,Gaze Pitch 0.778 elapsed:81470.4s
fold_path is still? /project/results/output/snapshots/fold14/
/project/results/output/snapshots/fold14/epoch_41.pkl
Taking snapshot...
Fold: 14 Epoch [42/60], Iter [42/420] Losses: Gaze Yaw 0.9903,Gaze Pitch 0.828 elapsed:81482.7s
Fold: 14 Epoch [42/60], Iter [84/420] Losses: Gaze Yaw 0.9897,Gaze Pitch 0.748 elapsed:81493.4s
Fold: 14 Epoch [42/60], Iter [126/420] Losses: Gaze Yaw 0.9248,Gaze Pitch 0.747 elapsed:81503.6s
Fold: 14 Epoch [42/60], Iter [168/420] Losses: Gaze Yaw 0.8983,Gaze Pitch 0.742 elapsed:81514.1s
Fold: 14 Epoch [42/60], Iter [210/420] Losses: Gaze Yaw 0.8989,Gaze Pitch 0.723 elapsed:81524.4s
Fold: 14 Epoch [42/60], Iter [252/420] Losses: Gaze Yaw 0.8761,Gaze Pitch 0.742 elapsed:815

Fold: 14 Epoch [49/60], Iter [84/420] Losses: Gaze Yaw 0.6612,Gaze Pitch 0.572 elapsed:82127.6s
Fold: 14 Epoch [49/60], Iter [126/420] Losses: Gaze Yaw 0.6926,Gaze Pitch 0.690 elapsed:82135.8s
Fold: 14 Epoch [49/60], Iter [168/420] Losses: Gaze Yaw 0.7213,Gaze Pitch 0.785 elapsed:82144.0s
Fold: 14 Epoch [49/60], Iter [210/420] Losses: Gaze Yaw 0.7141,Gaze Pitch 0.762 elapsed:82152.3s
Fold: 14 Epoch [49/60], Iter [252/420] Losses: Gaze Yaw 0.7115,Gaze Pitch 0.716 elapsed:82160.7s
Fold: 14 Epoch [49/60], Iter [294/420] Losses: Gaze Yaw 0.7072,Gaze Pitch 0.687 elapsed:82168.8s
Fold: 14 Epoch [49/60], Iter [336/420] Losses: Gaze Yaw 0.7128,Gaze Pitch 0.668 elapsed:82177.1s
Fold: 14 Epoch [49/60], Iter [378/420] Losses: Gaze Yaw 0.7363,Gaze Pitch 0.662 elapsed:82185.3s
Fold: 14 Epoch [49/60], Iter [420/420] Losses: Gaze Yaw 0.7483,Gaze Pitch 0.652 elapsed:82193.5s
fold_path is still? /project/results/output/snapshots/fold14/
/project/results/output/snapshots/fold14/epoch_49.pkl
Taking snaps

Fold: 14 Epoch [56/60], Iter [294/420] Losses: Gaze Yaw 0.6886,Gaze Pitch 0.561 elapsed:82808.1s
Fold: 14 Epoch [56/60], Iter [336/420] Losses: Gaze Yaw 0.6804,Gaze Pitch 0.563 elapsed:82816.4s
Fold: 14 Epoch [56/60], Iter [378/420] Losses: Gaze Yaw 0.6899,Gaze Pitch 0.554 elapsed:82824.8s
Fold: 14 Epoch [56/60], Iter [420/420] Losses: Gaze Yaw 0.6878,Gaze Pitch 0.558 elapsed:82833.3s
fold_path is still? /project/results/output/snapshots/fold14/
/project/results/output/snapshots/fold14/epoch_56.pkl
Taking snapshot...
Fold: 14 Epoch [57/60], Iter [42/420] Losses: Gaze Yaw 0.7682,Gaze Pitch 0.566 elapsed:82844.1s
Fold: 14 Epoch [57/60], Iter [84/420] Losses: Gaze Yaw 0.7680,Gaze Pitch 0.592 elapsed:82852.6s
Fold: 14 Epoch [57/60], Iter [126/420] Losses: Gaze Yaw 0.7395,Gaze Pitch 0.563 elapsed:82860.6s
Fold: 14 Epoch [57/60], Iter [168/420] Losses: Gaze Yaw 0.6983,Gaze Pitch 0.530 elapsed:82868.9s
Fold: 14 Epoch [57/60], Iter [210/420] Losses: Gaze Yaw 0.6835,Gaze Pitch 0.519 elapsed:828

In [13]:
!ls -l /project/results/output

total 4
drwxr-xr-x 17 root root 6144 Jun  8 01:15 snapshots


In [14]:
sys.exit()

SystemExit: 

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
!nvidia-smi

Wed Jun  8 02:52:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:10:1C.0 Off |                    0 |
| N/A   32C    P0    72W / 400W |  12270MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:10:1D.0 Off |                    0 |
| N/A   

In [16]:
!ls

Untitled.ipynb	train_test.ipynb


In [17]:
!pwd

/project/notebooks
